In [ ]:
import numpy as np

前向算法

1. **作用**：计算给定模型参数 $\lambda$ 时，观测序列 $O$ 出现的概率 $P(O|\lambda)$ 。  
    
2. **核心思想**：

    - 构建一个 $T \times N$ 的格栅（$T$ 为时间步，$N$ 为状态数）。

    - 每个节点 $\alpha(t,i)$ 表示：在时间 $t$ 处于状态 $i$ 并观测到 $o_1,o_2,...,o_t$ 的概率。  

3. **递推公式**：  

$$
\alpha(t,j) = \left[ \sum_{i=1}^N \alpha(t-1, i) \cdot a_{ij} \right] \cdot b_j(o_t)
$$

后向算法

1. **作用**：计算从时间 $t$ 到结束的观测概率 $P(o_{t+1},...,o_T | s_t=\theta_i)$ ，记为 $\beta_t(i)$ 。  

2. **递推公式**：

$$
\beta_t(i) = \sum_{j=1}^N a_{ij} \cdot b_j(o_{t+1}) \cdot \beta_{t+1}(j)
$$

3. **用途**：与前向算法结合用于参数学习（Baum-Welch算法）。

In [ ]:
class HMM:
    def __init__(self, states, observations, pi, A, B):
        self.states = states
        self.observations = observations
        self.pi = pi
        self.A = A
        self.B = B

    # 前向算法
    def forward(self, obs_seq):
        T = len(obs_seq)        # 序列长度
        N = len(self.states)    # 状态数
        alpha = np.zeros((T, N))
        
        # 初始化
        for i, q in enumerate(self.states):
            alpha[0][i] = self.pi[q] * self.B[q][obs_seq[0]]
        
        # 递推
        for t in range(1, T):
            for j, q_j in enumerate(self.states):
                alpha[t][j] = sum(
                    alpha[t-1][i] * self.A[self.states[i]][q_j] * self.B[q_j][obs_seq[t]]
                    for i, _ in enumerate(self.states)
                )
        
        total_prob = sum(alpha[T-1][i] for i in range(N))  # 计算总概率

        # 终止
        return alpha, total_prob
    
    # 后向算法
    def backward(self, obs_seq):
        T = len(obs_seq)
        N = len(self.states)
        beta = np.zeros((T, N))

        # 初始化：T时刻的beta值为1
        beta[T-1] = 1  # 索引从0开始，T-1对应第T个时刻

        # 递推：从T-2到0
        for t in range(T-2, -1, -1):
            for i, q_i in enumerate(self.states):
                beta[t][i] = sum(
                    self.A[q_i][self.states[j]] * 
                    self.B[self.states[j]][obs_seq[t+1]] * 
                    beta[t+1][j]
                    for j in range(N)
                )

        # 计算总概率（需结合初始状态和第一个观测）
        total_prob = sum(
            self.pi[self.states[i]] * 
            self.B[self.states[i]][obs_seq[0]] * 
            beta[0][i]
            for i in range(N)
        )

        return beta, total_prob

    
    # 维特比算法
    def viterbi(self, obs_seq):
        T = len(obs_seq)
        N = len(self.states)
        delta = np.zeros((T, N))
        psi = np.zeros((T, N), dtype=int)  # 回溯路径

        # 初始化
        for i, q in enumerate(self.states):
            delta[0][i] = self.pi[q] * self.B[q][obs_seq[0]]

        # 递推
        for t in range(1, T):
            for j, q_j in enumerate(self.states):
                max_val = -1
                max_idx = -1
                for i, q_i in enumerate(self.states):
                    val = delta[t-1][i] * self.A[q_i][q_j] * self.B[q_j][obs_seq[t]]
                    if val > max_val:
                        max_val = val
                        max_idx = i
                delta[t][j] = max_val
                psi[t][j] = max_idx

        # 回溯
        path = [np.argmax(delta[T-1])]
        for t in range(T-1, 0, -1):
            path.insert(0, psi[t][path[0]])

        return [self.states[idx] for idx in path]
    
    def baum_welch(self, obs_seq, max_iter=100):
        T = len(obs_seq)
        N = len(self.states)
        M = len(self.observations)

        for _ in range(max_iter):
            # E步: 计算 alpha, beta, gamma, xi
            alpha, _ = self.forward(obs_seq)
            beta, _ = self.backward(obs_seq)
            gamma = np.zeros((T, N))
            xi = np.zeros((T-1, N, N))

            for t in range(T-1):
                for i, q_i in enumerate(self.states):
                    for j, q_j in enumerate(self.states):
                        xi[t][i][j] = alpha[t][i] * self.A[q_i][q_j] * self.B[q_j][obs_seq[t+1]] * beta[t+1][j]
                xi[t] /= np.sum(xi[t])

            for t in range(T):
                gamma[t] = [alpha[t][i] * beta[t][i] for i in range(N)]
                gamma[t] /= np.sum(gamma[t])

            # M-step: 更新参数
            # 更新 pi
            for i, q in enumerate(self.states):
                self.pi[q] = gamma[0][i]

            # 更新 A
            for i, q_i in enumerate(self.states):
                denom = sum(gamma[t][i] for t in range(T-1))
                for j, q_j in enumerate(self.states):
                    self.A[q_i][q_j] = sum(xi[t][i][j] for t in range(T-1)) / denom

            # 更新 B
            for j, q_j in enumerate(self.states):
                denom = sum(gamma[t][j] for t in range(T))
                for k, o_k in enumerate(self.observations):
                    self.B[q_j][o_k] = sum(gamma[t][j] for t in range(T) if obs_seq[t] == o_k) / denom



In [ ]:
states = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]
